### Assignment 7 - Giulia Moret

##### In this assignment I have to fill the window table with the required data

In [33]:
import os
import pandas as pd
tables_folder = "/Users/giuliamoret/Documents/Documenti università/EETBS/CHARTS_AND_TABLES"
name_file_windows = "windows.csv"
path_file_windows = os.path.join(tables_folder,name_file_windows) 
window_DF = pd.read_csv(path_file_windows, sep=";", index_col = 0, header=0) 
print(window_DF)

               Direction  width  Height  Area Window_ID Frame_type  \
Name                                                                 
east                   E      8     1.8     0        5c      Fixed   
west                   W      8     1.8     0        5c      Fixed   
south-Fixed            S      2     1.8     0        5c      Fixed   
south-Operable         S      2     1.8     0        5c   Operable   

               Frame_material  U  SHGC  HF    ...     IAC_cl  IAC    Tx  Ed  \
Name                                          ...                             
east                     Wood  0     0   0    ...          0    0  1.00   0   
west                     Wood  0     0   0    ...          0    0  1.00   0   
south-Fixed              Wood  0     0   0    ...          0    0  1.00   0   
south-Operable           Wood  0     0   0    ...          0    0  0.64   0   

                ED PXI  FFs  C_value  CF  Qcooling  
Name                                               

In [34]:
window_DF.columns

Index([u'Direction', u'width', u'Height', u'Area', u'Window_ID', u'Frame_type',
       u'Frame_material', u'U', u'SHGC', u'HF', u'Q heating', u'SLF', u'Doh',
       u'Xoh', u'Fshd', u'IntShading_ID', u'IntShading_closeness', u'IAC_cl',
       u'IAC', u'Tx', u'Ed', u'ED', u'PXI', u'FFs', u'C_value', u'CF',
       u'Qcooling'],
      dtype='object')

#### Step 1) calculate the area

In [35]:
window_DF.loc[:,"Area"] = window_DF.loc[:,"Height"] * window_DF.loc[:,"width"]

#### Step 2) define a function that finds the shade line factor (SLF)

In [36]:
def SLF_finder(row):
    """I suppose that the latitude is 45"""
    SLF_table = "SLF.csv"
    path_SLF = os.path.join(tables_folder,SLF_table) 
    SLF_DF = pd.read_csv(path_SLF, sep=";", index_col = 0, header=0)
    direction = row["Direction"]
    SLF_value = SLF_DF.loc[direction,"45"]
    return SLF_value

window_DF.loc[:,"SLF"] = window_DF.apply(SLF_finder, axis=1)

#### Step 3) calculate the shaded fraction Fshd

In [37]:
calcolo = (window_DF.loc[:,"SLF"]*window_DF.loc[:,"Doh"]-window_DF.loc[:,"Xoh"])/window_DF.loc[:,"Height"]

In [38]:
window_DF.loc[:,"Fshd"] = calcolo
window_DF.loc[window_DF.loc[:,"Fshd"]>1,"Fshd"] = 1
window_DF.loc[window_DF.loc[:,"Fshd"]<0,"Fshd"] = 0

#### Step 4) define a function that reads IAC_cl from table

In [39]:
def IAC_cl_finder(row):
    window_ID = row["Window_ID"]
    int_shading_type = row["IntShading_ID"]
    IAC_cl_table = "IAC_cl.csv"
    path_IAC_cl = os.path.join(tables_folder,IAC_cl_table) 
    IAC_cl_DF = pd.read_csv(path_IAC_cl, sep=";", index_col = 1, header=0)
    value_IAC_cl = IAC_cl_DF.loc[window_ID,int_shading_type]
    return value_IAC_cl

window_DF.loc[:,"IAC_cl"] = window_DF.apply(IAC_cl_finder, axis=1)

#### Step 5) calculate IAC

In [40]:
window_DF.loc[:,"IAC"] = 1.0 + window_DF.loc[:,"IntShading_closeness"] * (window_DF.loc[:,"IAC_cl"] - 1)

#### Step 6) define two functions that read Ed and ED

In [41]:
def ED_finder(row):
    """I assume that the latitude is 45"""
    direction = row["Direction"]
    ED_table = "BeamIrradiance.csv"
    path_ED = os.path.join(tables_folder,ED_table) 
    ED_DF = pd.read_csv(path_ED, sep=";", index_col = 0, header=0)
    ED_value = ED_DF.loc[direction,"45"]
    return ED_value

def Ed_finder(row):
    """I assume that the latitude is 45"""
    direction = row["Direction"]
    Ed_table = "DiffuseIrradiance.csv"
    path_Ed = os.path.join(tables_folder,Ed_table) 
    Ed_DF = pd.read_csv(path_Ed, sep=";", index_col = 0, header=0)
    Ed_value = Ed_DF.loc[direction,"45"]
    return Ed_value

window_DF.loc[:,"Ed"] = window_DF.apply(Ed_finder, axis=1)
window_DF.loc[:,"ED"] = window_DF.apply(ED_finder, axis=1)

#### Step 7) PXI calculation

In [42]:
window_DF.loc[:,"PXI"] = window_DF.loc[:,"Tx"]*(window_DF.loc[:,"Ed"]+(1-window_DF.loc[:,"Fshd"])*window_DF.loc[:,"ED"])

#### Step 8) define a function that reads from the table the FFs value

In [43]:
def FFs_finder(row):
    """I assume that is a single family detached building"""
    FFs_table = "FFs.csv"
    path_FFs = os.path.join(tables_folder,FFs_table)
    FFs_DF = pd.read_csv(path_FFs, sep=";", index_col = 0, header=0)
    direction = row["Direction"]
    FFs_value = FFs_DF.loc[direction,"SingleFamilyDetached"]
    return FFs_value

window_DF.loc[:,"FFs"] = window_DF.apply(FFs_finder, axis=1)

In [46]:
name_of_modified_file = "window_gmoret.csv"
results_folder = "/Users/giuliamoret/Documents/Documenti università/EETBS/ASSIGNMENT"
path_file = os.path.join(results_folder,name_of_modified_file)
window_DF.to_csv(path_file, sep=";")